# Quick Start: Running Naive model on gift-eval benchmark

This notebook shows how to run the Naive model on the gift-eval benchmark.

Make sure you download the gift-eval benchmark and set the `GIFT-EVAL` environment variable correctly before running this notebook.

We will use the `Dataset` class to load the data and run the model. If you have not already please check out the [dataset.ipynb](./dataset.ipynb) notebook to learn more about the `Dataset` class. We are going to just run the model on two datasets for brevity. But feel free to run on any dataset by changing the `short_datasets` and `med_long_datasets` variables below.

In [1]:
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path=".env", encoding="utf-16")

# short_datasets = "m4_yearly m4_quarterly m4_monthly m4_weekly m4_daily m4_hourly electricity/15T electricity/H electricity/D electricity/W solar/10T solar/H solar/D solar/W hospital covid_deaths us_births/D us_births/M us_births/W saugeenday/D saugeenday/M saugeenday/W temperature_rain_with_missing kdd_cup_2018_with_missing/H kdd_cup_2018_with_missing/D car_parts_with_missing restaurant hierarchical_sales/D hierarchical_sales/W LOOP_SEATTLE/5T LOOP_SEATTLE/H LOOP_SEATTLE/D SZ_TAXI/15T SZ_TAXI/H M_DENSE/H M_DENSE/D ett1/15T ett1/H ett1/D ett1/W ett2/15T ett2/H ett2/D ett2/W jena_weather/10T jena_weather/H jena_weather/D bitbrains_fast_storage/5T bitbrains_fast_storage/H bitbrains_rnd/5T bitbrains_rnd/H bizitobs_application bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"
short_datasets = "m4_monthly"

# med_long_datasets = "electricity/15T electricity/H solar/10T solar/H kdd_cup_2018_with_missing/H LOOP_SEATTLE/5T LOOP_SEATTLE/H SZ_TAXI/15T M_DENSE/H ett1/15T ett1/H ett2/15T ett2/H jena_weather/10T jena_weather/H bitbrains_fast_storage/5T bitbrains_rnd/5T bizitobs_application bizitobs_service bizitobs_l2c/5T bizitobs_l2c/H"
med_long_datasets = ""

# Get union of short and med_long datasets
all_datasets = list(set(short_datasets.split() + med_long_datasets.split()))

dataset_properties_map = json.load(open("dataset_properties.json"))

In [2]:
from gluonts.ev.metrics import (
    MSE,
    MAE,
    MASE,
    MAPE,
    SMAPE,
    MSIS,
    RMSE,
    NRMSE,
    ND,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
# metrics = [
# MSE(forecast_type="mean"),
# MSE(forecast_type=0.5),
# MAE(),
# MASE(),
# MAPE(),
# SMAPE(),
# MSIS(),
# RMSE(),
# NRMSE(),
# ND(),
# MeanWeightedSumQuantileLoss(
#     quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# ),
# ]

metrics = [
    MSE(forecast_type="mean"),
    # MSE(forecast_type=0.5),
    MAE(forecast_type="mean"),
    MASE(forecast_type="mean"),
    MAPE(forecast_type="mean"),
    SMAPE(forecast_type="mean"),
    # MSIS(),
    RMSE(forecast_type="mean"),
    NRMSE(forecast_type="mean"),
    ND(forecast_type="mean"),
]

___
# MetaARIMA

In [3]:
import copy
import logging
from dataclasses import dataclass, field
from typing import Iterator, List, Optional

import numpy as np
import pandas as pd
from gluonts.core.component import validated
from gluonts.dataset import Dataset
from gluonts.dataset.util import forecast_start
from gluonts.model.forecast import QuantileForecast
from gluonts.model.predictor import RepresentablePredictor
from gluonts.transform.feature import LastValueImputation, MissingValueImputation
# from statsforecast import StatsForecast
# from statsforecast.models import SeasonalNaive


@dataclass
class _MetaConfig:
    quantile_levels: Optional[List[float]] = None
    forecast_keys: List[str] = field(init=False)

    def __post_init__(self):
        self.forecast_keys = ["mean"]
        if self.quantile_levels:
            self.forecast_keys += [f"{q:g}" for q in self.quantile_levels]


class MetaARIMAPredictor(RepresentablePredictor):
    ModelType = None

    @validated()
    def __init__(
        self,
        prediction_length: int,
        season_length: int,
        freq: str,
        quantile_levels: Optional[List[float]] = None,
        imputation_method: MissingValueImputation = LastValueImputation(),
        max_length: Optional[int] = None,
        batch_size: int = 1,
        parallel: bool = False,
        **model_params,
    ) -> None:
        super().__init__(prediction_length=prediction_length)
        self.freq = freq
        self.season_length = int(season_length)
        self.config = _MetaConfig(quantile_levels=quantile_levels)
        self.imputation_method = imputation_method

        self._base_model = model_params.get("base_model")
        if self._base_model is None:
            raise ValueError(
                "Missing required 'base_model' in **model_params. "
                "Provide a loaded MetaARIMA object supporting fit_model(df, freq) and predict(h)."
            )

        self.fallback_model = None
        # self.fallback_model = StatsForecast(
        #     models=[SeasonalNaive(season_length=self.season_length)],
        #     freq=freq,
        #     n_jobs=-1 if parallel else 1,
        # )

        self.logger = logging.getLogger(__name__)
        if not self.logger.handlers:
            logging.basicConfig(level=logging.INFO)

    def _to_long_df(self, entry: dict) -> pd.DataFrame:
        start = entry["start"]
        target = np.asarray(entry["target"], np.float32)
        if np.isnan(target).any():
            target = self.imputation_method(target)
        uid = f"{entry['item_id']}_{str(forecast_start(entry)).replace(' ', 'T')}"
        freq_str = entry.get("freq", start.freq)
        return pd.DataFrame(
            {
                "unique_id": uid,
                "ds": pd.date_range(
                    start=start.to_timestamp(),
                    periods=len(target),
                    freq=freq_str,
                ).to_numpy(),
                "y": target,
            }
        )

    def predict(self, dataset: Dataset, **kwargs) -> Iterator[QuantileForecast]:
        self.logger.info("MetaARIMA: single-origin per series (full-context fit).")
        last_by_item: dict[str, dict] = {}
        order_index: dict[str, int] = {}
        for i, entry in enumerate(dataset):
            item_id = entry["item_id"]
            last_by_item[item_id] = entry
            order_index[item_id] = i
        for item_id in sorted(order_index.keys(), key=lambda k: order_index[k]):
            entry = last_by_item[item_id]
            df = self._to_long_df(entry)
            try:
                model = copy.deepcopy(self._base_model)
                model.fit(df, seas_length=self.season_length, freq=self.freq)
                f_df = model.predict(h=self.prediction_length)
                if "MetaARIMA" in f_df.columns:
                    yhat = f_df["MetaARIMA"].to_numpy()
                else:
                    yhat = f_df.select_dtypes(include=[np.number]).iloc[:, 0].to_numpy()
            except Exception as e:
                self.logger.info(
                    f"MetaARIMA failed for {df['unique_id'].iloc[0]}: {e}. Falling back."
                )
                fb = self.fallback_model.forecast(df=df, h=self.prediction_length)
                col = next(c for c in fb.columns if c.startswith("SeasonalNaive"))
                yhat = fb[col].to_numpy()
            if (yhat.shape[0] != self.prediction_length) or np.isnan(yhat).any():
                fb = self.fallback_model.forecast(df=df, h=self.prediction_length)
                col = next(c for c in fb.columns if c.startswith("SeasonalNaive"))
                yhat = fb[col].to_numpy()
            start_date = forecast_start(entry)
            forecast_arrays = np.stack([yhat] * len(self.config.forecast_keys), axis=0)
            yield QuantileForecast(
                forecast_arrays=forecast_arrays,
                forecast_keys=self.config.forecast_keys,
                start_date=start_date,
                item_id=df["unique_id"].iloc[0],
            )

## Evaluation

Now that we have our predictor class, we can use it to predict on the gift-eval benchmark datasets. We will use the `evaluate_model` function to evaluate the model. This function is a helper function to evaluate the model on the test data and return the results in a dictionary. We are going to follow the naming conventions explained in the [README](../README.md) file to store the results in a csv file called `all_results.csv` under the `results/naive` folder.

The first column in the csv file is the dataset config name which is a combination of the dataset name, frequency and the term:

```python
f"{dataset_name}/{freq}/{term}"
```


In [4]:
import logging
logger = logging.getLogger("MetaARIMA")
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(name)s: %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)

In [5]:
from gluonts.model import evaluate_model
import csv
import os
import time
from gluonts.time_feature import get_seasonality
from gift_eval.data import Dataset

os.environ["NIXTLA_ID_AS_COL"] = "1"

# Iterate over all available datasets

output_dir = "../results/metaarima"
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "all_results.csv")


pretty_names = {
    "saugeenday": "saugeen",
    "temperature_rain_with_missing": "temperature_rain",
    "kdd_cup_2018_with_missing": "kdd_cup_2018",
    "car_parts_with_missing": "car_parts",
}


with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    # writer.writerow(
    #     [
    #         "dataset",
    #         "model",
    #         "eval_metrics/MSE[mean]",
    #         "eval_metrics/MSE[0.5]",
    #         "eval_metrics/MAE[0.5]",
    #          "eval_metrics/MASE[0.5]",
    #          "eval_metrics/MAPE[0.5]",
    #          "eval_metrics/sMAPE[0.5]",
    #          "eval_metrics/MSIS",
    #         "eval_metrics/RMSE[mean]",
    #         "eval_metrics/NRMSE[mean]",
    #         "eval_metrics/ND[0.5]",
    #         "eval_metrics/mean_weighted_sum_quantile_loss",
    #         "domain",
    #         "num_variates",
    #     ]
    # )

    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MAE[mean]",
            "eval_metrics/MASE[mean]",
            "eval_metrics/MAPE[mean]",
            "eval_metrics/sMAPE[mean]",
            #  "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[mean]",
            "domain",
            "num_variates",
        ]
    )

# for ds_name in all_datasets:
#     print(f"Processing dataset: {ds_name}")

#     terms = ["short", "medium", "long"]
#     for term in terms:
#         if (
#             term == "medium" or term == "long"
#         ) and ds_name not in med_long_datasets.split():
#             continue

#         if "/" in ds_name:
#             ds_key = ds_name.split("/")[0]
#             ds_freq = ds_name.split("/")[1]
#             ds_key = ds_key.lower()
#             ds_key = pretty_names.get(ds_key, ds_key)
#         else:
#             ds_key = ds_name.lower()
#             ds_key = pretty_names.get(ds_key, ds_key)
#             ds_freq = dataset_properties_map[ds_key]["frequency"]
#         ds_config = f"{ds_key}/{ds_freq}/{term}"

#         # Initialize the dataset
#         to_univariate = (
#             False
#             if Dataset(name=ds_name, term=term, to_univariate=False).target_dim == 1
#             else True
#         )
#         dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate)
#         test_split_iter = dataset.test_data
#         test_data = next(iter(test_split_iter))


In [ ]:
from gluonts.model import evaluate_model
import csv
import os
import time
from gluonts.time_feature import get_seasonality
from gift_eval.data import Dataset

from metaarima.metaarima_loader import load_metaarima_model

metaarima_model = load_metaarima_model(
    # windows
    "assets/trained_metaarima_m4_monthly_catboost.joblib.gz"
    # mac OS
    # "//Users/ricardoinacio/projects/experiments-metaarima/assets/trained_metaarima_m4_yearly_catboost.joblib.gz"
    # "//Users/ricardoinacio/projects/experiments-metaarima/assets/trained_metaarima_m4_monthly_catboost.joblib.gz"
    # "//Users/ricardoinacio/projects/experiments-metaarima/assets/trained_metaarima_m4_monthly_catboost.joblib.gz"
)

os.environ["NIXTLA_ID_AS_COL"] = "1"

# Iterate over all available datasets

output_dir = "../results/metaarima"
# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "all_results.csv")

pretty_names = {
    "saugeenday": "saugeen",
    "temperature_rain_with_missing": "temperature_rain",
    "kdd_cup_2018_with_missing": "kdd_cup_2018",
    "car_parts_with_missing": "car_parts",
}

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    # writer.writerow(
    #     [
    #         "dataset",
    #         "model",
    #         "eval_metrics/MSE[mean]",
    #         "eval_metrics/MSE[0.5]",
    #         "eval_metrics/MAE[0.5]",
    #          "eval_metrics/MASE[0.5]",
    #          "eval_metrics/MAPE[0.5]",
    #          "eval_metrics/sMAPE[0.5]",
    #          "eval_metrics/MSIS",
    #         "eval_metrics/RMSE[mean]",
    #         "eval_metrics/NRMSE[mean]",
    #         "eval_metrics/ND[0.5]",
    #         "eval_metrics/mean_weighted_sum_quantile_loss",
    #         "domain",
    #         "num_variates",
    #     ]
    # )

    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MAE[mean]",
            "eval_metrics/MASE[mean]",
            "eval_metrics/MAPE[mean]",
            "eval_metrics/sMAPE[mean]",
            #  "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[mean]",
            "domain",
            "num_variates",
        ]
    )

for ds_name in all_datasets:
    print(f"Processing dataset: {ds_name}")

    terms = ["short", "medium", "long"]
    for term in terms:
        if (
            term == "medium" or term == "long"
        ) and ds_name not in med_long_datasets.split():
            continue

        if "/" in ds_name:
            ds_key = ds_name.split("/")[0]
            ds_freq = ds_name.split("/")[1]
            ds_key = ds_key.lower()
            ds_key = pretty_names.get(ds_key, ds_key)
        else:
            ds_key = ds_name.lower()
            ds_key = pretty_names.get(ds_key, ds_key)
            ds_freq = dataset_properties_map[ds_key]["frequency"]
        ds_config = f"{ds_key}/{ds_freq}/{term}"

        # Initialize the dataset
        to_univariate = (
            False
            if Dataset(name=ds_name, term=term, to_univariate=False).target_dim == 1
            else True
        )
        dataset = Dataset(name=ds_name, term=term, to_univariate=to_univariate)
        season_length = get_seasonality(dataset.freq)

        print(f"freq: {dataset.freq}, season length: {season_length}")

        # if "A" in str(dataset.freq):
        if "M" in str(dataset.freq):
            predictor = MetaARIMAPredictor(
                prediction_length=dataset.prediction_length,
                season_length=season_length,  # or the integer seasonal period you need
                freq=dataset.freq,           # e.g. "M"
                base_model=metaarima_model,
                quantile_levels=None,
                batch_size=1,
                logger=logger
            )

            # Measure the time taken for evaluation
            res = evaluate_model(
                predictor,
                test_data=dataset.test_data,
                metrics=metrics,
                batch_size=1,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )
            print(sorted(k for k in res.keys() if isinstance(k, str)))

            # Append the results to the CSV file
            with open(csv_file_path, "a", newline="") as csvfile:
                writer = csv.writer(csvfile)
                # writer.writerow(
                #     [
                #         ds_config,
                #         "metaarima",
                #         res["MSE[mean]"][0],
                #         res["MSE[0.5]"][0],
                #         res["MAE[0.5]"][0],
                #         res["MASE[0.5]"][0],
                #         res["MAPE[0.5]"][0],
                #         res["sMAPE[0.5]"][0],
                #         res["MSIS"][0],
                #         res["RMSE[mean]"][0],
                #         res["NRMSE[mean]"][0],
                #         res["ND[0.5]"][0],
                #         res["mean_weighted_sum_quantile_loss"][0],
                #         dataset_properties_map[ds_key]["domain"],
                #         dataset_properties_map[ds_key]["num_variates"],
                #     ]
                # )

                writer.writerow(
                    [
                        ds_config,
                        "metaarima",
                        res["MSE[mean]"][0],
                        # res["MSE[0.5]"][0],
                        res["MAE[mean]"][0],
                        res["MASE[mean]"][0],
                        res["MAPE[mean]"][0],
                        res["sMAPE[mean]"][0],
                        # res["MSIS"][0],
                        res["RMSE[mean]"][0],
                        res["NRMSE[mean]"][0],
                        res["ND[mean]"][0],
                        # res["mean_weighted_sum_quantile_loss"][0],
                        dataset_properties_map[ds_key]["domain"],
                        dataset_properties_map[ds_key]["num_variates"],
                    ]
                )

            print(f"Results for {ds_name} have been written to {csv_file_path}")

Processing dataset: m4_monthly
freq: M, season length: 12


0it [00:00, ?it/s]INFO:__main__:MetaARIMA: single-origin per series (full-context fit).
11251it [7:21:21,  2.15s/it]

## Results

Running the above cell will generate a csv file called `all_results.csv` under the `results/naive` folder containing the results for the Naive model on the gift-eval benchmark. The csv file will look like this:


In [ ]:
import pandas as pd

df = pd.read_csv("../results/metaarima/all_results.csv")
df

In [ ]:
df = pd.read_csv("../results/auto_arima/all_results.csv")
df[df["dataset"].isin(["hospital/M/short"])]